# We need to make a table of well locations and abstraction/injection rates to provide to our building process using `modflow-setup`. Typically, these would be provided independently, but we will create them from the model made with pure `flopy` earlier.

In [ ]:
# first we need to import all the modules we will use
import sys, os
import pandas as pd
sys.path.append('../../dependencies/')
import flopy as fp

### for MODFLOW 6 in `flopy`, we load the simulation and then run the `get_model()` method to return the model. We will be able to pull the well information from the groundwater flow model. Note that this step is required becuase a single MODFLOW 6 simulation can contain multiple models

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws = '../../models/sgn_model_3layer/',
                               load_only=['wel'])

In [ ]:
# here is the groundwater flow model
gwf = sim.get_model()

### let's confirm that there is only one stress period of well data defined - the `stress_period_data` data contains a dictionary, so we cen inspect the keys

In [ ]:
# yep - only one stress period (steady state)
gwf.wel.stress_period_data.data.keys()

In [ ]:
gwf.wel.stress_period_data.data[0]

### the data are stored in a `numpy` `recarray` object but it is often easier to work with a `pandas` `DataFrame` object, so we can convert it

In [ ]:
spd = pd.DataFrame.from_records(gwf.wel.stress_period_data.data[0])
spd.tail()

### the `gwf` object contains a stack of arrays with the model top and all layer bottoms in it, so we can use that to determine where each well should be placed vertically

In [ ]:
gwf.modelgrid.top_botm

### we are going to use a bunch of list comprehension constructions, so here is a quick example and a [link](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions) to more information on how they work in python #powerful!

In [ ]:
[i**3 for i in range(5)]

In [ ]:
spd['layer'] = [i[0] for i in spd.cellid]
spd['cell'] = [i[1] for i in spd.cellid]
spd['laytop'] = gwf.modelgrid.top_botm[spd.layer,spd.cell]
spd['laybot'] = gwf.modelgrid.top_botm[spd.layer+1,spd.cell]
spd['laymidpt'] = spd.laytop-(spd.laytop - spd.laybot)/2
spd

### there's some encoded extra text in the boundnames (`\x00`) that we can simply strip off each name

In [ ]:
spd.iloc[0].boundname

In [ ]:
spd['rootname'] = [i.replace('\x00','').replace("well-","") for i in spd.boundname]

In [ ]:
spd.iloc[0].rootname

In [ ]:
spd.dtypes

### finally, let's add the layer number to each name in the `boundname` column

In [ ]:
spd.boundname = [f'{i.rstrip()}__{lay}' for i,lay in zip(spd.rootname,spd.layer)]

In [ ]:
spd

### ...and save this out to a CSV file for use with `modflow-setup`

In [ ]:
spd.to_csv('../../data/sgn/wells_with_elev.dat')